In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ROOT='''/content/drive/MyDrive/Colab Notebooks/googledrive/colorDetection'''

Mounted at /content/drive


In [ ]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [ ]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_azure','h_histogram','h_google']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Catch'}[pagename.split('_')[1]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'b':'azure','c':'histogram','d':'google'}[pagename.split('_')[1]]

conn = sqlite3.connect(ROOT+"/Feedback_Color/ec2_db/turing_feedback_color.db")
cursor = conn.cursor()
cursor.execute("select * from tablechairDB;")

label_dict = {'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'histogram':1,'google':1,'catch':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'catch':'CatchTrial','azure':'Azure','histogram':'Histogram','google':'Google'}

count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))
dummy_marks_overall = []
for i in range(len(all_data)):
    if all_data[i][15] in [5]:
#         print(all_data[i][11])
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 100
        threshold = nTrials*(7/12) # change to number of total trials
        for dat in data_dict['data']:

            if dat['trialdata']['phase'] == 'TEST':
                present = dat['trialdata']['oneWord_curr'] == '0'
                response_present = dat['trialdata']['topic'] == '1'
                if present == response_present:
                    dummy_mark += 1
        if dummy_mark < threshold:
            continue
        dummy_marks_overall.append(dummy_mark)

        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0


        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':

                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']


#                 trials_dict['Trial_'+str(count)] = trial_data
#                 trial_data = trial_data['trialData']
                trial_data['trialData']['correct response'] = get_gt_from_pagename(trial_data['trialData']['page_name'])
                trial_data['trialData']['choose response'] = trial_data['trialData']['Aclass']

#                 print(trial_data)
                if trial_data['trialData']['correct response'] == 'Catch':
                    catch_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    catch_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif trial_data['trialData']['correct response'] == 'Swapped':
                    swap_trial_mat[0][label_dict[trial_data['trialData']['choose response']]] += 1
                    swap_trial_mat_overall[0][label_dict[trial_data['trialData']['choose response']]] += 1
                elif nTrial > 40:
                    confmatrices['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    confmat_overall['h_machine'][label_dict[trial_data['trialData']['correct response']]][label_dict[trial_data['trialData']['choose response']]] += 1
                    if trial_data['trialData']['correct response'] == "Machine":
                        model = get_model_from_pagename(trial_data['trialData']['page_name'])
                        confmatrices['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                        confmat_overall['h_{}'.format(model)][0][label_dict[trial_data['trialData']['choose response']]] += 1
                    marks.append(1 if trial_data['trialData']['correct response'] == trial_data['trialData']['choose response'] else 0)
                    marks_conf.append(deepcopy(confmatrices['h_machine']))

#                 trial_data['trialData'] = dat['trialdata']
                if trial_data['trialData']['correct response']  in ['Catch','Swapped'] or nTrial <=40:
                    continue

                count += 1
                trial_data['trialData']['response_gender'] = trial_data['trialData']['Agender']
                del trial_data['trialData']['Agender']

                trial_data['trialData']['response_speaker'] = trial_data['trialData']['choose response'].lower()
                del trial_data['trialData']['choose response']

                trial_data['trialData']['response_object'] = trial_data['trialData']['topic'].lower()
                del trial_data['trialData']['topic']

                trial_data['trialData']['groundtruth'] = trial_data['trialData']['correct response'].lower()


                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['oneWord_curr'].lower()
                del trial_data['trialData']['oneWord_curr']

                if trial_data['trialData']['correct response'] in ["Machine","Catch","Swapped"]:
                    trial_data['trialData']['machine_groundtruth'] = modelname_dict[model.lower()]
                else:
                    trial_data['trialData']['machine_groundtruth'] = ''

                del trial_data['trialData']['correct response']
#                 trial_data['trialData']["imgtype"] = "naturaldesign"



                trials_dict['Trial_Color_'+str(count)] = trial_data

#         plot_learning_curve_single_subject(marks_conf,participant_count)
        print(confmatrices)
        print('catch',catch_trial_mat)
#         print('swap',swap_trial_mat)
#     else:
#         print(all_data[i][15])
conn.close()

[Finished Date] 2025-03-28 11:04:57
[Time Used] 21.9 mins
[Questionnaire] {'country': 'United States', 'age': '51', 'engagement': '6', 'difficulty': '8', 'mode': 'live', 'gender': 'male', 'education': 'bachelor', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 93/100
{'h_machine': array([[20.,  9.],
       [13., 13.]]), 'h_azure': array([[4., 3.]]), 'h_histogram': array([[6., 7.]]), 'h_google': array([[3., 3.]])}
catch [[0. 5.]]
[Finished Date] 2025-03-28 10:52:55
[Time Used] 9.5 mins
[Questionnaire] {'country': 'United States', 'age': '36-40', 'engagement': '9', 'difficulty': '10', 'mode': 'live', 'gender': 'female', 'education': 'high school', 'ai-experience': '1', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 89/100
{'h_machine': array([[15., 15.],
       [12., 12.]]), 'h_azure': array([[4., 4.]]), 'h_histogram': array([[6., 6.]]), 'h_google': array([[2., 2.]])}
catch [[0. 6.]]
[Finished Date] 2025-03-28 11:09:06
[Tim

In [ ]:
with open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Plot/Data_all/humanjudge_Color_Feedback_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

In [ ]:
np.mean(dummy_marks_overall)

np.float64(88.72)

In [ ]:
catch_trial_mat_overall

array([[ 70., 271.]])

In [ ]:
confmat_overall

{'h_machine': array([[867., 617.],
        [505., 670.]]),
 'h_azure': array([[172., 237.]]),
 'h_histogram': array([[154., 247.]]),
 'h_google': array([[179., 186.]])}

In [ ]:
len(trials_dict)/len(dummy_marks_overall)

53.18

In [ ]:
len(dummy_marks_overall)

50

In [ ]:
import torch
def sample_to_imbedding(pagename,gt):
    with open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Feedback_Color/static/responses_color/{}".format(trial_data['trialData']['page_name'][:-1]), 'r') as f:
        raw_text = f.readlines()[0]
    color = raw_text.split('Speaker Response: ')[-1]
    color = color.split('</h4>')[0]
    color_imbbedding = color_to_embedding[color.lower()]
    img_imbbedding = image_imbedding_from_pagename(pagename)
    return np.array(torch.hstack((img_imbbedding, color_imbbedding[0])).detach()).flatten(), {'human':0,'machine':1}[gt.lower()]


x,y = sample_to_imbedding(trial_data['trialData']['page_name'],trial_data['trialData']['groundtruth'])
x.shape

(1280,)

In [ ]:
trial_data['trialData']

{'rt': 5129,
 'Aclass': 'Machine',
 'imageID': 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/65.jpg',
 'trial': 100,
 'page_name': '65_d_0.html/',
 'phase': 'TEST',
 'response_gender': 'NA',
 'response_speaker': 'machine',
 'response_object': '1',
 'groundtruth': 'machine',
 'object_groundtruth': '0',
 'machine_groundtruth': 'Google',
 'imgtype': 'naturaldesign'}